# Adding an agent

In this section, we will add a new agent. We will continue to use the files from the previous tutorial where we added `solarPV` and created a new scenario. We will start by creating a new agent `A2` with identical properties to the existing agent `A1`, before modifying the investment strategies of the new agent.

To achieve this, first, we must modify the ```Agents.csv``` file in the directory:

    {PATH_TO_MODEL}/technodata/Agents.csv

To start with, we will copy the data from the first two rows of agent `A1`, creating a new agent `A2` with identical objectives to `A1`. 

Also notice that we amend the `Quantity` column. The reason for this is that we want to specify that Agent A1 makes up 50% of the population, and A2 makes up the remaining 50% of the population.

Again, we only show some of the columns due to space constraints, however see [here](https://github.com/SGIModel/MUSE_OS/blob/main/docs/tutorial-code/2-add-agent/1-single-objective/technodata/Agents.csv) for the full file.

|AgentShare|Name|RegionName|Objective1|Objective2|Objective3|ObjData1|ObjData2|…|DecisionMethod|Quantity|…|Type|
|-|-|-|-|-|-|-|-|-|-|-|-|-|
|Agent1|A1|R1|LCOE|||1||…|singleObj|**0.5**|…|New|
|Agent2|A1|R1|LCOE|||1||…|singleObj|**0.5**|…|Retrofit|
|**Agent3**|**A2**|**R1**|**LCOE**|||**1**||…|**singleObj**|**0.5**|…|**New**|
|**Agent4**|**A2**|**R1**|**LCOE**|||**1**||…|**singleObj**|**0.5**|…|**Retrofit**|

We then edit all of the `Technodata` files to split the existing capacity between the two agents by the proportions we like. As we now have two agents which take up 50% of the population each, we will split the existing capacity by 50% for each of the agents. Notice that we only require the columns `Agent2` and `Agent4` to define the retrofit agents.

The new technodata file for the power sector will look like the following:

|ProcessName|RegionName|Time|Level|cap_par|cap_exp|…|Fuel|EndUse|Agent2|Agent4|
|-|-|-|-|-|-|-|-|-|-|-|
|Unit|-|Year|-|MUS$2010/PJ_a|-|…|-|-|Retrofit|**Retrofit**|
|gasCCGT|R1|2020|fixed|23.78234399|1|…|gas|electricity|**0.5**|**0.5**|
|gasCCGT|R1|2040|fixed|23.78234399|1|…|gas|electricity|**0.5**|**0.5**|
|windturbine|R1|2020|fixed|36.30771182|1|…|wind|electricity|**0.5**|**0.5**|
|windturbine|R1|2040|fixed|36.30771182|1|…|wind|electricity|**0.5**|**0.5**|
|solarPV|R1|2020|fixed|40|1|...|solar|electricity|**0.5**|**0.5**|
|solarPV|R1|2040|fixed|30|1|...|solar|electricity|**0.5**|**0.5**|

However, remember you will have to make the same changes for the residential and gas sectors!

We will now save this file and run the new simulation model using the following command:

    python -m muse settings.toml

Again, we use seaborn and pandas to analyse the data in the ```Results``` folder.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
mca_capacity = pd.read_csv(
    "../tutorial-code/2-add-agent/1-single-objective/Results/MCACapacity.csv"
)
mca_capacity.head()

This time we can see that there is data for the new agent, `A2`. Next, we will visualise the investments made by each of the agents using seaborn's facetgrid command.

In [ ]:
mca_capacity = pd.read_csv(
    "../tutorial-code/2-add-agent/1-single-objective/Results/MCACapacity.csv"
)

for sector_name, sector_data in mca_capacity.groupby("sector"):
    sector_capacity = (
        sector_data.groupby(["year", "agent", "technology"]).sum().reset_index()
    )
    g = sns.FacetGrid(data=sector_capacity, col="agent")
    g.map_dataframe(
        lambda data, **kwargs: data.pivot(
            index="year", columns="technology", values="capacity"
        ).plot(kind="bar", stacked=True, ax=plt.gca())
    )
    g.add_legend()
    g.set_ylabels("Capacity (PJ)")
    g.figure.suptitle(f"{sector_name.capitalize()} Sector:")
    g.figure.subplots_adjust(top=0.8)

In this scenario, with both agents running the same objective, identical results between agents can be seen. There is large uptake in `windturbine`, whilst `solarPV` is invested in especially in later years where a change in the price of `solarPV` was set in the previous tutorial.

Next, we will see what occurs if the agents invest based upon different investment strategies. We will modify `A2` to make investments based upon a mixture of [levelised cost of electricity (LCOE)](https://en.wikipedia.org/wiki/Levelized_cost_of_energy) and [equivalent annual cost (EAC)](https://en.wikipedia.org/wiki/Equivalent_annual_cost). These two objectives will be combined by calculating a weighted sum of the two when comparing potential investment options. We will give both LCOE and EAC a relative weight value of 0.5.

To do this, we need to modify the `Objective2`, `ObjData1`, `ObjData2`, `Objsort2` and `DecisionMethod` columns in the ```Agents.csv``` file, to look like the following:

|AgentShare|Name|RegionName|Objective1|Objective2|Objective3|ObjData1|ObjData2|…|Objsort2|…|DecisionMethod|Quantity|…|Type|
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
|Agent1|A1|R1|LCOE|||1||…||…|singleObj|0.5|…|New|
|Agent2|A1|R1|LCOE|||1||…||…|singleObj|0.5|…|Retrofit|
|Agent3|A2|R1|LCOE|**EAC**||**0.5**|**0.5**|…|**True**|**…**|**weighted_sum**|0.5|…|New|
|Agent4|A2|R1|LCOE|**EAC**||**0.5**|**0.5**|…|**True**|**…**|**weighted_sum**|0.5|…|Retrofit|

The `Objsort2` parameter indicates that the new objective should be minimised, rather than maximised.

We will then re-run the simulation. You may find that the simulation fails to complete due to insufficient capacity limits in the `residential` sector. In this case, try doubling `MaxCapacityGrowth` for `gasboiler` and `heatpump` in the `technodata/residential/Technodata.csv` file to 0.04 and running the simulation again.

Once the model has run to completion, we can visualise the results as before:

In [ ]:
mca_capacity = pd.read_csv(
    "../tutorial-code/2-add-agent/2-multiple-objective/Results/MCACapacity.csv"
)

for sector_name, sector_data in mca_capacity.groupby("sector"):
    sector_capacity = (
        sector_data.groupby(["year", "agent", "technology"]).sum().reset_index()
    )
    g = sns.FacetGrid(data=sector_capacity, col="agent")
    g.map_dataframe(
        lambda data, **kwargs: data.pivot(
            index="year", columns="technology", values="capacity"
        ).plot(kind="bar", stacked=True, ax=plt.gca())
    )
    g.add_legend()
    g.set_ylabels("Capacity (PJ)")
    g.figure.suptitle(f"{sector_name.capitalize()} Sector:")
    g.figure.subplots_adjust(top=0.8)

In this scenario we can see two divergent strategies. Both agents invest heavily in `windturbine` early on, and relatively little in `gasCCGT`. The share of `solarPV` increases in later years as the price falls (as set in the previous tutorial), but this increase is more dramatic for agent `A2`.

From this small scenario, the difference between investment strategies between agents is evident. This is one of the key benefits of agent-based models when compared to optimisation-based models.

Have a play around with the files to see if you can come up with different scenarios!

## Next steps 

In the next section we will show you how to add a new region. The next tutorial will build on the model from tutorial 1 (with just one agent), so the model from this tutorial can be left aside for now.